In [362]:
import numpy as np

In [363]:
def ingresar_matriz():
    # Solicitar las dimensiones de la matriz
    filas = int(input("Ingresa el número de filas: "))
    columnas = int(input("Ingresa el número de columnas: "))

    print(f"Por favor, introduce las filas de la matriz con los elementos separados por espacio:")
    matriz = []

    # Recopilar las filas de la matriz
    for i in range(filas):
        while True:
            try:
                fila = list(map(float, input(f"Fila {i + 1}: ").split()))
                if len(fila) != columnas:
                    raise ValueError(f"La fila debe tener exactamente {columnas} elementos.")
                matriz.append(fila)
                break
            except ValueError as e:
                print(f"Error: {e}. Intenta de nuevo.")

    return matriz

In [364]:
A = np.array(ingresar_matriz())

Ingresa el número de filas: 2
Ingresa el número de columnas: 2
Por favor, introduce las filas de la matriz con los elementos separados por espacio:
Fila 1: 2 1
Fila 2: 3 4


In [365]:
type(A)

numpy.ndarray

In [366]:
np.poly(A)#determinante de (A-I\landa)

array([ 1., -6.,  5.])

In [367]:
B = 1 + max(np.abs(np.poly(A)/np.poly(A)[0])) #calculo de la cota de cauchy-lagrage para polinomios
B

7.0

In [368]:
def evaluar(pol, x): #definiion de la funciòn evalur para polinomios
  pol = pol.copy()
  n = len(pol) - 1
  return sum([pol[i] * x ** (n-i) for i in range(n+1)])

In [369]:
#criterio de strums#
P = []#sucesión de polinomios
n = A.shape[0]#dimensiòn de la matriz por tanto numero de iteraciones d ela sucesión
polinomio_original = np.poly(A)#determinante de (A-I\landa)
polinomio_polarizado = np.asarray([polinomio_original[i] * B ** (n - i) for i in range(n + 1)]) #cambio de variable#
P.append(polinomio_polarizado)#P_0 primer tèrmino de la sucesión
P.append(np.array([P[0][i] * (n-i) for i in range(0, n)]))#P_1 segundo término de la sucesión

for i in range(2, n + 1):
  P.append(-np.polynomial.polynomial.polydiv(P[i-2], P[i-1])[1])#descripción


In [370]:
for pol in P:
  print(evaluar(pol, -1))

96.0
-140.0
21.777777777777786


In [371]:
#evaluar la lista P en 0#
for pol in P:
    pol[-1]

In [372]:
#evaluar la lista P en 1#
for pol in P:
    np.sum(pol)

In [373]:
#conteo de cambios de signo en la lsita p(-1)#
conteo_0 = 0
for i in range(1, n+1):
  if evaluar(P[i-1], -1) * evaluar(P[i],-1) < 0:
    conteo_0 += 1
print(conteo_0)

2


In [374]:
#conteo de cambios de signo para la lista p(1)#
conteo_1=0
for i in range(1,n+1):
  if evaluar(P[i-1], 1)*evaluar(P[i],1) < 0:
    conteo_1 += 1
print(conteo_1)

0


In [375]:
N=conteo_0-conteo_1 #numero de raices
print(N)

2


In [376]:
def conteo_raices(intervalo):
  (a, b) = intervalo
  conteo_a = 0 #conteo de los cambios de signos en cada uno de los intervalos dados por la particion en la lista dada por el teorema de strums evaluada en los limites inferiores de cada subintervalo dada por la particiones
  for i in range(1, n+1):
    if evaluar(P[i-1], a) * evaluar(P[i],a) < 0:
      conteo_a += 1

  conteo_b = 0 #conteo de los cambios de signos en cada uno de los intervalos dados por la particion
  for i in range(1,n+1):
    if evaluar(P[i-1], b)*evaluar(P[i],b) < 0:
      conteo_b += 1

  return conteo_a - conteo_b #numero de raices en los intervalos encontrados en la partición#

In [377]:
max_raices = conteo_raices((-1, 1))
n1 = 0
while max_raices > 1:
  k = 2 ** n1
  intervalos = [(-1 + j / k, - 1 + (j + 1) / k) for j in range(2 * k)] #particion del intervalo [-1,1]#

  max_raices = 0
  for intervalo in intervalos:

    N = conteo_raices(intervalo) #conteo de raices en cada subintervalo definido por la partción

    if N > max_raices:
      max_raices = N #localización de la raices en las particiones del intervalo[-1,1]
  n1 += 1


In [378]:
#aislamiento de raices#
verdaderos_intervalos = []
for intervalo in intervalos:
  if conteo_raices(intervalo) > 0:
    verdaderos_intervalos.append(intervalo)

In [379]:
verdaderos_intervalos

[(0.0, 0.5), (0.5, 1.0)]

In [380]:
#reprametrización de los intervalos donde se encuentaran las raices en el intervalos [-B,B]
for intervalo in verdaderos_intervalos:
  print((intervalo[0] * B, intervalo[1] * B))

(0.0, 3.5)
(3.5, 7.0)


In [381]:
def biseccion(intervalo, tol): #Función, extremos, y tolerancia
  (a, b) = intervalo
  p = a + ( b - a ) / 2 #Punto medio del intervalo
  if evaluar(P[0], p) == 0 or b-a < tol:
    return p
  elif evaluar(P[0], a) * evaluar(P[0], p) <0:
    return biseccion((a, p),tol)
  elif evaluar(P[0], p) * evaluar(P[0], b) <0:
    return biseccion((p, b),tol)

In [382]:
#encontrar las raices del polinomio parametrizado usando el método de bisección#
raices = []
for intervalo in verdaderos_intervalos:
  raices.append(biseccion(intervalo, 10**(-10)))

In [383]:
valores_propios = [B * raiz for raiz in raices]#reparametrización de las raíces ubicadas en [-B,B]
valores_propios

[1.0000000000873115, 5.000000000029104]

In [384]:
#calculo de los vectores propios por gauss seidel#
import numpy as np

def gradiente_conjugado(A, b, x0=None, tol=1e-10, max_iter=1000):

    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()  # Vector inicial (usualmente 0)
    r = b - np.dot(A, x)  # Resíduo inicial
    p = r.copy()  # Dirección de búsqueda inicial

    for k in range(max_iter):
        # Producto escalar de p con r using .flatten() to ensure 1D arrays for dot product
        # This ensures the dot product calculates a scalar as intended.
        alpha = np.dot(r.flatten(), r.flatten()) / np.dot(p.flatten(), np.dot(A, p).flatten())

        # Actualización de la solución
        x = x + alpha * p

        # Nuevo residuo
        r_new = r - alpha * np.dot(A, p)

        # Comprobar la convergencia
        if np.linalg.norm(r_new) < tol:
            print(f"Convergió después de {k+1} iteraciones.")
            return x

        # Calculamos beta using .flatten() to ensure 1D arrays for dot product
        beta = np.dot(r_new.flatten(), r_new.flatten()) / np.dot(r.flatten(), r.flatten())

        # Actualizamos la dirección
        p = r_new + beta * p

        r = r_new  # Actualizamos el residuo

    print("Alcanzado el número máximo de iteraciones.")
    return x

In [385]:
#solucion del sistema de ecuaciones para cada valor propio
for valor in valores_propios:
  #sistema de ecuaciones para cada valor propio
  A_hat = A - valor * np.eye(n) #matriz (A-\landa I)
  b = np.zeros((n,1)) #vector 0 de dimension nx1
  x0 = np.ones((n,1))#valor de inicio para el método gauss seidel en este caso la identidad
  x = gradiente_conjugado(A_hat,b,x0, 1e-4,100)#solución del sistema de ecuaciones (A-\landa I)x=0
  print(f"El autovector asociado a {valor} es {x}")

Convergió después de 1 iteraciones.
El autovector asociado a 1.0000000000873115 es [[ 0.5]
 [-0.5]]
Convergió después de 1 iteraciones.
El autovector asociado a 5.000000000029104 es [[0.5]
 [1.5]]
